In [46]:
import os
import shutil
import codecs
import chardet
import json

def run_txt(input_dir, character):
    """
    Renames game generated .run files into .txt files

    Args:
        input_dir (dir): dir with .run files
        character (str): character name
    """
    run = 1
    for file in os.listdir(input_dir):
        os.rename(f'{input_dir}{file}', f'{input_dir}{character}{run}.txt')
        run += 1


def run_tojson(input_dir, output_file):
    """
    Input all run.txt files into json file

    Args:
        input_dir (dir): dir with run.txt files
        output_file (file): empty json file
    """
    for file in os.listdir(input_dir):
        with open(f'{input_dir}{file}', 'r') as input_file:
            run_dict = input_file.readlines()
            with open(output_file, 'a') as output_json:
                json.dump(run_dict, output_json)

run_tojson('./watcher/', './watcher_runs.json')


In [114]:
def seed_del(input_dir):
    """
    Finds unecessary data and deletes

    Args:
        input_dir (dir): dir with .run files
    """
    for file in os.listdir(input_dir):
        with open(file, 'r+') as cleantxt_file:
            for line in cleantxt_file:
                fileContent = re.sub(',"special_seed":0,', ',', line)
                cleantxt_file.write(fileContent)

seed_del('./ironclad/')

FileNotFoundError: [Errno 2] No such file or directory: 'ironclad18.txt'

In [91]:
def run_totsv(input_dir, output_file):
    """
    Converts run.txt files into TSV files

    Args:
        input_dir (dir): dir with run.txt files
        output_file (file): tsv file
    """
    for file in os.listdir(input_dir):
        with open(f'{input_dir}{file}', 'r') as input_file:
            run_list = input_file.readlines()
            for i in run_list:
                run_dict = json.loads(i)
                tsv_columns = run_dict.keys()
                tsv_data = '  '.join(tsv_columns) + '\n'
                new_row = list()
                for col in tsv_columns:
                    new_row.append(str(run_dict[col]))
                tsv_data += '  '.join(new_row) + '\n'
                with open(output_file, 'a') as out_tsv:
                    out_tsv.write(tsv_data)


run_totsv('./silent/', './silent_runs.tsv')

In [93]:
import re

def to_csv(input_tsv, output_csv): 
    """
    Converts TSV file to CSV file

    Args:
        input_tsv (file): tsv file
        output_csv (file): csv file
    """
    with open(input_tsv, 'r') as tsv_file:  
        with open(output_csv, 'w') as csv_file:
            for line in tsv_file:
                fileContent = re.sub("  ", ",", line)
                csv_file.write(fileContent)

to_csv('silent_runs.tsv', 'silent.csv')

In [146]:
import pandas as pd
import datetime
from google.cloud import bigquery
import pandas
import pytz

# def add_dataframes():

cols = ['gold_per_floor','floor_reached','playtime','items_purged','score','play_id','local_time','is_ascension_mode','campfire_choices','neow_cost','seed_source_timestamp','circlet_count','master_deck','relics','potions_floor_usage','damage_taken','seed_played','potions_obtained','is_trial','path_per_floor','character_chosen','items_purchased','campfire_rested','item_purchase_floors','current_hp_per_floor','gold','neow_bonus','is_prod','is_daily','chose_seed','campfire_upgraded','win_rate','timestamp','path_taken','build_version','purchased_purges','victory','max_hp_per_floor','card_choices','player_experience','relics_obtained','event_choices','is_beta','boss_relics','items_purged_floors','is_endless','potions_floor_spawned','killed_by','ascension_level','extra']
defect_df = pd.read_csv('CSVData/defect.csv', names=cols)
watcher_df = pd.read_csv('CSVData/watcher.csv', names=cols)
silent_df = pd.read_csv('CSVData/silent.csv', names=cols)
ironclad_df = pd.read_csv('CSVData/ironclad.csv', names=cols)

all_chars_df_mess = pd.concat([defect_df,watcher_df,silent_df,ironclad_df])

all_chars_df = all_chars_df_mess.drop_duplicates()

completed_run_df = all_chars_df[all_chars_df.floor_reached != 0]

completed_run_df

client = bigquery.Client(project= 'deb-01-346001')
table = 'STS_run_data.april_runs'
job_config = bigquery.LoadJobConfig()
job = client.load_table_from_dataframe(completed_run_df, table, job_config=job_config)